# Fine-Tuning Supervisado con SFTTrainer

Este notebook demuestra cómo realizar fine-tuning del modelo `HuggingFaceTB/SmolLM2-135M` usando el `SFTTrainer` de la librería `trl`. Las celdas del notebook se ejecutan y harán el fine-tuning del modelo. Puedes seleccionar tu nivel de dificultad probando diferentes conjuntos de datos.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Ejercicio: Fine-Tuning de SmolLM2 con SFTTrainer</h2>
    <p>Toma un conjunto de datos desde el repositorio de Hugging Face y realiza un fine-tuning de un modelo con él.</p>
    <p><b>Niveles de dificultad</b></p>
    <p>🐢 Usa el conjunto de datos `HuggingFaceTB/smoltalk`</p>
    <p>🐕 Prueba el conjunto de datos `bigcode/the-stack-smol` y realiza un fine-tuning de un modelo de generación de código en un subconjunto específico `data/python`.</p>
    <p>🦁 Selecciona un conjunto de datos que esté relacionado con un caso de uso del mundo real que te interese.</p>
</div>

In [ ]:
# Instalar los requisitos en Google Colab
# !pip install transformers datasets trl huggingface_hub


# Autenticación en Hugging Face
from huggingface_hub import login

login()

# Para mayor comodidad, puedes crear una variable de entorno con tu token de Hugging Face como HF_TOKEN en tu archivo .env

In [ ]:
# Importa las bibliotecas necesarias
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

# Establece dinámicamente el dispositivo de procesamiento.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Carga el modelo y el tokenizador
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Configura el formato de chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Establece nuestro nombre para guardar y/o subir el modelo finetuneado
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generar con el modelo base

Aquí probaremos el modelo base, que no tiene una plantilla de chat.

In [ ]:
# Probemos el modelo base antes de entrenar
prompt = "Write a haiku about programming"

# Formatea con la plantilla
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Genera la respuesta
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Preparación del Conjunto de Datos

Cargaremos un conjunto de datos de ejemplo y lo formatearemos para el entrenamiento. El conjunto de datos debe estar estructurado con pares de entrada-salida, donde cada entrada es un prompt y la salida es la respuesta esperada del modelo.

**TRL formateará los mensajes de entrada según las plantillas de chat del modelo.** Estos deben representarse como una lista de diccionarios con las claves: `role` y `content`.


In [ ]:
# Carga un conjunto de datos de ejemplo
from datasets import load_dataset

# TODO: define tu conjunto de datos y configuración usando los parámetros de ruta y nombre
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 Si tu conjunto de datos no está en un formato que TRL pueda convertir a la plantilla de chat, necesitarás procesarlo. Consulta el [módulo](../chat_templates.md)

## Configuración del SFTTrainer

El `SFTTrainer` se configura con varios parámetros que controlan el proceso de entrenamiento. Estos incluyen el número de pasos de entrenamiento, el tamaño del lote, la tasa de aprendizaje y la estrategia de evaluación. Ajusta estos parámetros según tus requisitos específicos y recursos computacionales.

In [ ]:
# Configura el SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",  # Directorio de salida para los resultados del entrenamiento
    max_steps=1000,  # Ajusta según el tamaño del conjunto de datos y la duración de entrenamiento deseada
    per_device_train_batch_size=4,  # Ajusta según la capacidad de la memoria de tu GPU
    learning_rate=5e-5,  # Punto de partida común para el fine-tuning
    logging_steps=10,  # Frecuencia de registro de métricas de entrenamiento
    save_steps=100,  # Frecuencia de guardado de puntos de control del modelo
    evaluation_strategy="steps",  # Evalua el modelo en intervalos regulares
    eval_steps=50,  # Frecuencia de evaluación
    use_mps_device=(
        True if device == "mps" else False
    ),  # Usa MPS para entrenamiento con precisión mixta
    hub_model_id=finetune_name,  # Asigna un nombre único a tu modelo
)

# Inicializa el SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 ajusta los parámetros del SFTTrainer a tu conjunto de datos elegido. Por ejemplo, si usas el conjunto de datos `bigcode/the-stack-smol`, necesitarás elegir la columna `content`.


## Entrenamiento del Modelo

Con el entrenador configurado, ahora podemos proceder a entrenar el modelo. El proceso de entrenamiento implicará iterar sobre el conjunto de datos, calcular la pérdida y actualizar los parámetros del modelo para minimizar esta pérdida.

In [ ]:
# Entrena el modelo
trainer.train()

# Guarda el modelo
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Ejercicio adicional: Generar con el modelo ajustado</h2>
    <p>🐕 Utiliza el modelo ajustado para generar una respuesta, de la misma forma que con el ejemplo base.</p>
</div>


In [ ]:
# Prueba el modelo ajustado con el mismo mensaje

# Probemos el modelo base antes del entrenamiento
prompt = "Escribe un haiku sobre programación"

# Formatea con la plantilla
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Genera respuesta
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: usa el modelo ajustado para generar una respuesta, igual que en el ejemplo base.

## 💐 ¡Has terminado!

Este notebook proporcionó una guía paso a paso para realizar fine-tuning del modelo `HuggingFaceTB/SmolLM2-135M` utilizando el `SFTTrainer`. Siguiendo estos pasos, puedes adaptar el modelo para realizar tareas específicas de manera más efectiva. Si deseas continuar trabajando en este curso, aquí tienes algunas sugerencias que podrías intentar:

- Intenta este notebook en un nivel de dificultad más alto.
- Revisa el PR de un compañero.
- Mejora el material del curso a través de un Issue o PR.
